# Automatic Speech Recognition (ASR) Tutorial 

## Fine-tune a pretrained, multilingual ASR model on FLEURS

In this tutorial, we will be evaluating and improving a multilingual ASR model for a language in the FLEURS dataset. We will focus on **Hausa**, but you can follow along in any language in FLEURS. See the FLEURS dataset paper for a list of supported languages: https://arxiv.org/abs/2205.12446.

We will be looking at three major open-source ASR multilingual models:
* XLS-R  (https://arxiv.org/abs/2111.09296)
* Whisper (https://cdn.openai.com/papers/whisper.pdf)
* MMS (https://scontent-sjc3-1.xx.fbcdn.net/v/t39.8562-6/348827959_6967534189927933_6819186233244071998_n.pdf?_nc_cat=104&ccb=1-7&_nc_sid=ad8a9d&_nc_ohc=-JOSFMsFL-UAX-4O6o4&_nc_ht=scontent-sjc3-1.xx&oh=00_AfDdMFq0DP2xIRyjWpGrmIpqncnouiylLfWnFsAgxboLWw&oe=6497E242)

## Before you start: Setting up your coding environment 

You can run follow along and run the lines of code in this notebook, and also utilize the scripts found in this GitHub respository. Before starting this tutorial, you will need to create a virtual environment for this project so you can download all the required packages without affecting your other projects. We recommend using Anaconda (conda) to create a virtual environment. We have provided an `environment.yml` file that you can use to create a virtual environment named `asr` containing all the required packages. In your terminal run this code:

```
git clone https://github.com/kashrest/lrl-asr-experiments.git
cd lrl-asr-experiments
conda env create -f environment.yml 
conda activate asr
```

Note: The pretrained multilingual ASR models we will be using in this notebook require GPUs with at least 40 GB of space (check) for practical use. 

Now you can run the lines in this notebook

## Data Preprocessing

The first step is to download and prepare the data for the ASR model. Hugging Face has an easy way to download FLEURS data for any supported language, where the split can be specified

In [1]:
import os 
from datasets import load_dataset

cache_dir_fleurs = "./data/fleurs/"
out_dir = "./asr-tutorial-experiment-1/"

# create a data directory for caching 
try:
    os.mkdir(cache_dir_fleurs)
except:
    pass

# create a directory for outputs 
try:
    os.mkdir(out_dir)
except:
    pass
    
# for Hausa, the language code is "ha_ng"
train_data = load_dataset("google/fleurs", "ha_ng", split="train", cache_dir=cache_dir_fleurs)
val_data = load_dataset("google/fleurs", "ha_ng", split="validation", cache_dir=cache_dir_fleurs)
test_data = load_dataset("google/fleurs", "ha_ng", split="test", cache_dir=cache_dir_fleurs)

Found cached dataset fleurs (/data/users/kashrest/lrl-asr-experiments/data/fleurs/google___fleurs/ha_ng/2.0.0/af82dbec419a815084fa63ebd5d5a9f24a6e9acdf9887b9e3b8c6bbd64e0b7ac)
Found cached dataset fleurs (/data/users/kashrest/lrl-asr-experiments/data/fleurs/google___fleurs/ha_ng/2.0.0/af82dbec419a815084fa63ebd5d5a9f24a6e9acdf9887b9e3b8c6bbd64e0b7ac)
Found cached dataset fleurs (/data/users/kashrest/lrl-asr-experiments/data/fleurs/google___fleurs/ha_ng/2.0.0/af82dbec419a815084fa63ebd5d5a9f24a6e9acdf9887b9e3b8c6bbd64e0b7ac)


FLEURS data is organized like so

In [2]:
train_data[0]

{'id': 302,
 'num_samples': 301440,
 'path': '/data/users/kashrest/asr-experiments/downloads/extracted/953575415c600d2042020b042380119265aefaa4fcf95afc300adfcd2b79784e/10002175198254707815.wav',
 'audio': {'path': 'train/10002175198254707815.wav',
  'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         6.78300858e-05, 1.26361847e-05, 6.46114349e-05]),
  'sampling_rate': 16000},
 'transcription': 'nasarorin da vautier ta samu marasa alaka da bada umarni ba sun hada da yajin cin abinci a 1973 a kan abin da ya ke ganin dabaibayin siyasa ne',
 'raw_transcription': 'Nasarorin da Vautier ta samu marasa alaka da bada umarni ba sun hada da yajin cin abinci a 1973 a kan abin da ya ke ganin dabaibayin siyasa ne.',
 'gender': 1,
 'lang_id': 30,
 'language': 'Hausa',
 'lang_group_id': 3}

We are interested in the audio (represented as an array of floats each describing the amplitude/loudness of the sound; the number of floats is determined by the sampling rate which here is 16,000 measurements per second) and the corresponding transcript

In [3]:
train_transcripts, val_transcripts, test_transcripts = [], [], []
train_audio, val_audio, test_audio = [], [], []

for elem in train_data:
    train_audio.append(elem["audio"]["array"])
    train_transcripts.append(elem["raw_transcription"])
    
for elem in val_data:
    val_audio.append(elem["audio"]["array"])
    val_transcripts.append(elem["raw_transcription"])
    
for elem in test_data:
    test_audio.append(elem["audio"]["array"])
    test_transcripts.append(elem["raw_transcription"])

Now, since we are interested in transcribing speech, we want to clean the transcripts by removing special characters that do not have a clear sound (such as ! '). This part may depend on your target application and language. For example for Hausa, many native speakers do not speak English and does not have much code-switching, so we want to also normalize any foreign characters (ç ş) and symbols (% & $).

In [4]:
import re
def preprocess_texts_hausa(transcriptions):
    chars_to_remove_regex = '[\,\?\!\-\;\:\"\“\%\‘\'\ʻ\”\�\$\&\(\)\–\—]'

    def _remove_special_characters(transcription):
        transcription = transcription.strip()
        transcription = transcription.lower()
        transcription = re.sub(chars_to_remove_regex, '', transcription)
        transcription = re.sub("\[\]\{\}", '', transcription)
        transcription = re.sub(r'[\\]', '', transcription)
        transcription = re.sub(r'[/]', '', transcription)
        transcription = re.sub(u'[¥£°¾½²]', '', transcription)
        transcription = re.sub(u'[\+><]', '', transcription)
        return transcription

    def _normalize_diacritics(transcription):
        a = '[āăáã]'
        u = '[ūúü]'
        o = '[öõó]' 
        c = '[ç]'
        i = '[í]'
        s = '[ş]'
        e = '[é]'

        transcription = re.sub(a, "a", transcription)
        transcription = re.sub(u, "u", transcription)
        transcription = re.sub(o, "o", transcription)
        transcription = re.sub(c, "c", transcription)
        transcription = re.sub(i, "i", transcription)
        transcription = re.sub(s, "s", transcription)
        transcription = re.sub(e, "e", transcription)

        return transcription

    cleaned_transcriptions = map(_remove_special_characters, transcriptions)
    cleaned_transcriptions = list(map(_normalize_diacritics, list(cleaned_transcriptions)))
    return cleaned_transcriptions

train_transcripts = preprocess_texts_hausa(train_transcripts)
val_transcripts = preprocess_texts_hausa(val_transcripts)
test_transcripts = preprocess_texts_hausa(test_transcripts)

Some models predict one character at a time, and so we need a character vocabulary made up of all characters in the dataset after preprocessing. We can save the vocabulary in a JSON file

In [5]:
import json 

def extract_all_chars(transcription):
      all_text = " ".join(transcription)
      vocab = list(set(all_text))
      return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = list(map(extract_all_chars, train_transcripts))
vocab_val = list(map(extract_all_chars, val_transcripts))
vocab_test = list(map(extract_all_chars, test_transcripts))

vocab_train_chars = []
for elem in [elem["vocab"][0] for elem in vocab_train]:
    vocab_train_chars.extend(elem)

vocab_val_chars = []
for elem in [elem["vocab"][0] for elem in vocab_val]:
    vocab_val_chars.extend(elem)

vocab_test_chars = []
for elem in [elem["vocab"][0] for elem in vocab_test]:
    vocab_test_chars.extend(elem)

vocab_list = list(set(vocab_train_chars) | set(vocab_val_chars) | set(vocab_test_chars))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}

# for word delimiter, change " " --> "|" (ex. "Hello my name is Bob" --> "Hello|my|name|is|Bob")
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict) # this is for CTC to predict the end of a character (e.g. "hhh[PAD]iii[PAD]iii[PAD]" == "hii")

with open(out_dir+"vocab_hausa.json", 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

## Evaluation code

In ASR, word error rate (WER) and character error rate (CER) are the common metrics used to evaluate how good a model-produced transcript is in comparison to the gold transcript.

In [6]:
from datasets import load_metric, load_dataset, Audio
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")
                         
def compute_metrics(label_strs, pred_strs):
    # make sure labels are preprocessed the same way for proper comparison with other models after finetuning
    wer = wer_metric.compute(predictions=pred_strs, references=label_strs) * 100
    cer = cer_metric.compute(predictions=pred_strs, references=label_strs) * 100
    return {"wer": wer, "cer": cer}

def compute_metrics_finetuning(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str) * 100
    cer = cer_metric.compute(predictions=pred_str, references=label_str) * 100
    return {"wer": wer, "cer": cer}

/tmp/ipykernel_24443/2987210431.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


## Section A: Zero-Shot ASR

Let's run inference on our audio dataset with different existing, ready-to-use pretrained and finetuned models. Later, when we compare model performance, we will compare performance on the test set since some models will be trained on the train split.

### OpenAI Whisper

OpenAI's Whisper model is a pretrained encoder-decoder model that supports a set of languages without futher fine-tuning. Here, we will use whisper-small for example reasons. You can use the larger checkpoints for better performance, all which can be found on Hugging Face Hub: https://huggingface.co/openai/whisper-small.

Note: Whisper requires that input is sampled at 16,000 Hz. FLEURS data is sampled at this rate, so we are good.

In [22]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from tqdm import tqdm
import time

device = "cuda:0" # change this to a gpu if you have access to one, otherwise set to "cpu"
model_id = "openai/whisper-large-v2" # 8630MiB for batch size 1, 25426MiB for batch size 10
processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(model_id).to(device)
forced_decoder_ids = processor.get_decoder_prompt_ids(language="hausa", task="transcribe")

predicted_test_transcripts = []

batch_size = 100
z = (len(test_audio)//batch_size)+(len(test_audio)%batch_size)

for i in tqdm(range(0, z+batch_size+1, batch_size)):
    print(i)
    continue
    batch = test_audio[i:i+batch_size] if i+batch_size <= len(test_audio) else test_audio[i:]
    input_features = processor(batch, sampling_rate=16000, return_tensors="pt").input_features.to(device)
    # generate token ids
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    # decode token ids to text
    predicted_test_transcripts.extend(processor.batch_decode(predicted_ids, skip_special_tokens=True))

100%|███████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 35246.25it/s]

0
100


In [14]:
len(predicted_test_transcripts), len(test_audio)

(80, 621)

Let's evaluate the performance of whisper-small on our preprocessed test dataset

In [ ]:
compute_metrics(test_transcripts, predicted_test_transcripts)

It looks like we have a 102.6% WER and 42.2% CER. Let's create a running table of the performances of different models on our test dataset. 

| Model | WER | CER|
|-------|-----|----|
|whisper-small| 102.6| 42.2|


### Facebook MMS

In [24]:
from transformers import Wav2Vec2ForCTC, AutoProcessor

model_id = "facebook/mms-1b-all"

processor = AutoProcessor.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)

processor.tokenizer.set_target_lang("hau")
model.load_adapter("hau")

predicted_test_transcripts = []

for elem in tqdm(test_audio): # Todo: batch inference?
    inputs = processor(elem, sampling_rate=16_000, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs).logits
    ids = torch.argmax(outputs, dim=-1)[0]
    predicted_test_transcripts.append(processor.decode(ids))

  0%|                                                                                        | 0/621 [00:00<?, ?it/s]


NameError: name 'torch' is not defined

In [ ]:
compute_metrics(test_transcripts, predicted_test_transcripts)

## Section B: Finetuning

### Whisper

### MMS

### XLS-R

## Section C: Further Improvements

### Hyperparameter tuning

### Adding more data